In [1]:
from postgres_connect import *

In [2]:
path = 'tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('Yes',path)

Tunnel Started
Connection Made


In [3]:
query_path = 'rider_pip_closure.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
cursor = conn.cursor()
cursor.execute(custom_query)
conn.commit()
# conn.close()

In [4]:
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)

In [5]:
df

,key,pip_date,rider_id,shipping_city,rider_name,hub,vendor_name,rider_age,need_improvement,starting_fasr,closing_fasr,pip_status,pip_result,update_time,starting_productivity,closing_productivity
0,20240212-DS_BOM_KRL_28062022_003,2024-02-12,DS_BOM_KRL_28062022_003,Mumbai,Salauddin Shaikh,DS BOM KRL,Ambika Logistics,207,Morning Prepaid (76%) Morning COD(0%) Producti...,58.0,91.0,closed,failed,2024-02-20 23:17:20.301878,9.4,7.66666666666667
1,20240212-DS_NCT_DWRK_28032022_055,2024-02-12,DS_NCT_DWRK_28032022_055,Delhi,Jaipal,DS DEL DWK,PS Manpower,207,Morning COD(25%) Productivity (7),50.0,NaN,closed,failed,2024-02-20 23:17:20.301878,7.0,None
2,20240212-DS_NCT_DWRK_28032022_99,2024-02-12,DS_NCT_DWRK_28032022_99,Delhi,Naveen Kumar,DS DEL DWK,PS Manpower,207,Morning COD(52%),80.0,84.0,closed,passed,2024-02-20 23:17:20.301878,24.571428571428573,27
3,20240212-Ennamalla_8309541054_04,2024-02-12,Ennamalla_8309541054_04,Hyderabad,Ennamalla satyanarayana,DS HYD TRNK,Blitz_Self,47,Morning Prepaid (79%),75.0,75.0,closed,failed,2024-02-20 23:17:20.301878,18.0,16.7142857142857
4,20240212-Faheem_7039724062_03,2024-02-12,Faheem_7039724062_03,Mumbai,Faheem shaikh,DS BOM KRL,Ambika Logistics,55,Morning Prepaid (77%) Morning COD(40%) Product...,73.0,93.0,closed,failed,2024-02-20 23:17:20.301878,8.0,6.28571428571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,20240212-Vijay_kumar_7,2024-02-12,Vijay_kumar_7,Bangalore,Bhanu Chander G,DS BLR HBBL,Blitz_Self,149,Morning Prepaid (44%) Productivity (13),81.0,81.0,closed,failed,2024-02-20 23:17:20.301878,13.285714285714286,14
79,20240212-Yadhu_9986435484_02,2024-02-12,Yadhu_9986435484_02,Bangalore,Yadhu MS,DS BLR BOMM,Quess Works,16,Productivity (9),82.0,90.0,closed,failed,2024-02-20 23:17:20.301878,9.285714285714286,0
80,20240212-Yahiya_7406458114_02,2024-02-12,Yahiya_7406458114_02,Bangalore,Yahiya,DS BLR BOMM,Blitz_Self,13,Productivity (13),75.0,77.0,closed,failed,2024-02-20 23:17:20.301878,12.714285714285714,14.8571428571429
81,20240212-Yahya_Iqtedar_husain__3,2024-02-12,Yahya_Iqtedar_husain__3,Mumbai,Yahya Iqtedar husain,DS BOM KRL,Team HR,92,Morning Prepaid (71%) Morning Prepaid (33%) Pr...,60.0,47.0,closed,failed,2024-02-20 23:17:20.301878,8.666666666666666,4.5


In [6]:
query_path = 'rider_pip_closure_performance.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
df_pip = pd.read_sql(custom_query,conn)
df_pip.head()

,pip_date,shipping_city,rider_id,rider_name,hub,performance_type,threshold,monday,tuesday,wednesday,thursday,friday,saturday,sunday,avg_fasr,status
0,2024-02-12,Bangalore,Ananda_8660619030_02,Ananda M,DS BLR BOMM,Morning-Prepaid,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail
1,2024-02-12,Bangalore,Ananda_8660619030_02,Ananda M,DS BLR BOMM,Morning-COD,0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail
2,2024-02-12,Bangalore,Ananda_8660619030_02,Ananda M,DS BLR BOMM,Evening-Prepaid,0.80,1.0,0.77,1.0,1.00,1.0,0.66,0.87,0.87,pass
3,2024-02-12,Bangalore,Ananda_8660619030_02,Ananda M,DS BLR BOMM,Evening-COD,0.50,0.0,1.00,1.0,0.33,0.5,0.00,1.00,0.46,fail
4,2024-02-12,Bangalore,Habeeb_8095475952_02,Habeeb Ulla,DS BLR BOMM,Morning-Prepaid,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail


In [9]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Filter failed riders
failed_riders = df[df['pip_result'] == 'failed']

# Group by city and hub
grouped_data = failed_riders.groupby(['shipping_city', 'hub']).size().reset_index(name='failed_count')

# Group by city to get total rider count
total_riders = df.groupby(['shipping_city', 'hub']).size().reset_index(name='total_count')

# Merge the two dataframes on city
result_df = pd.merge(total_riders, grouped_data, on=['shipping_city','hub'], how='left')

# Calculate the failed rate without decimals
result_df['failed_rate'] = (result_df['failed_count'] / result_df['total_count']) * 100
result_df['failed_rate'] = result_df['failed_rate'].fillna(0)
result_df['failed_rate'] = result_df['failed_rate'].astype(int)

# Email configuration
sender_email = "akshay@blitznow.in"
sender_password = "Outofblitz1!"

# List of recipients
recipients = [
    "akshay@blitznow.in",
    "sanjay@blitznow.in", 
    "david.kumar@blitznow.in", 
    "ajay.verma@blitznow.in", 
    "biswojit.jena@blitznow.in", 
    "chandrakanth.sunkoji@blitznow.in", 
    "rahul.lochav@blitznow.in", 
    "sourabh.vijay@blitznow.in", 
    "sourabh.vijayvergia@blitznow.in", 
    "syed.afrid@blitznow.in", 
    "anandbellary.hindushree@blitznow.in", 
    "dilip.kumar@blitznow.in", 
    "ma.saifullah@blitznow.in", 
    "shankar.poddar@blitznow.in", 
    "sumit.kamble@blitznow.in", 
    "anil@blitznow.in"
]

# Create the message
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = ", ".join(recipients)
message['Subject'] = f"Rider PIP Closure Report Week - {failed_riders['pip_date'].max()}"

# Create an Excel file with two sheets
excel_file_path = 'failed_riders_report.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    # Write the failed_riders DataFrame to the first sheet
    failed_riders.to_excel(writer, sheet_name='Failed_Riders_Data', index=False)

    # Write the df_pip DataFrame to the second sheet
    df_pip.to_excel(writer, sheet_name='Additional_Sheet', index=False)

# Attach the Excel file
with open(excel_file_path, 'rb') as excel_file:
    attachment = MIMEApplication(excel_file.read(), _subtype='xlsx')
    attachment.add_header('Content-Disposition', 'attachment', filename='pip_closure_report.xlsx')
    message.attach(attachment)

# Create an HTML table for the email body with formatted headers
html_table = result_df.to_html(index=False, border=1, classes='table', escape=False, justify='center')
html_table = html_table.replace('<th>', '<th style="font-weight:bold; background-color:lightblue; text-align:center;">').replace('_', ' ').upper()

# Add the table to the email body
body = f"""\
<html>
  <head></head>
  <body>
    <p>Dear recipients,</p>
    <p>Please find attached the report for RIDER PIP Result for the week - {failed_riders['pip_date'].max()}.</p>
    <p>Summary:</p>
    {html_table}
    <p>Best regards,<br>Your Name</p>
  </body>
</html>
"""

message.attach(MIMEText(body, 'html'))

# Connect to the SMTP server and send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipients, message.as_string())

print("Email sent successfully.")

Email sent successfully.


In [16]:
conn.close()